# Data, classification task

Our data will be a set of articles downloaded from Wikipedia that we need to classify as concerning 'Medical' topics, or 'Non medical'; in particular, we need to classify documents in two classes: those with a 'Medical' tags, and those without it.

## Data retrieval

To create a dataset that can be used for the classification task we need to download articles from Wikipedia API. Wikipedia already groups articles under different categories, so for this binary classification task we can just use their functions to select a category and the download articles belonging to that category.

Wikipedia categories are highly hierarchical, so, for e.g., the category 'Medicine' contains few articles and many subcategories, which are themselves articles. To 'fix' the search I did some exploratory analysis of the available categories to determine which had enouigh articles to build a decently-sized dataset. 

For the medical catogories, I went with the following categories:

In [3]:
medical_categories = [
    "Category:Alternative medicine stubs",
    "Category:Evidence-based medicine",
    "Category:Veterinary medicine stubs",
    "Category:Vaccination",
    "Category:2018 disease outbreaks",
]

For the non-medical categories, I went with scientific and non-scientific categories: . I did explicity add scientific/related categories to make the classification task more interesting: i expect an article belonging to the 'Literature' to be classified correctly quite easily; an article belonging to 'Geology' or 'AI', not so much (due to possible overlapping topics).  

In [4]:
non_medical_categories = [
    "Category:Dark ages",
    "Category:Historiography of China",
    "Category:Sports controversies",
    "Category:Philosophy of artificial intelligence",
    "Category:Geology",
    "Category:Space",
    "Category:Literature",
    "Category:Music videos",
]